In [1]:

from utils.prompt import template
from utils.utils import process_pdfs_from_dataframe, process_pdfs_from_dataframe_faiss

import pandas as pd
import string


## Dense DB (Vector DB) 및 Sparse DB

test.csv에서 질문들에 매칭되는 소스(pdf) 정보가 주어진다. 

각 질문에 따른 소스에 접근을 하고 리트리버를 사용해도 되기 때문에 

각 질문에 따른 소스별 DB를 만들어주자

In [2]:
df = pd.read_csv('../open/test.csv')
base_directory = '../open' # Your Base Directory

df.head()

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?


In [3]:
source = list(df['Source'].unique())

# 알파벳 리스트
alphabet = list(string.ascii_uppercase)

In [4]:
# 딕셔너리로 파일명과 알파벳 매핑
file_mapping = {alphabet[i]: source[i] for i in range(len(source))}

reversed_file_mapping = {value: key for key, value in file_mapping.items()}

In [5]:
# PDF별 FAISS db 생성 
import os 
save_path = 'data'
os.makedirs(save_path, exist_ok=True)
process_pdfs_from_dataframe_faiss(df, base_directory, save_path, reversed_file_mapping)

가지고 있는 pdf파일들: ['./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf'
 './test_source/보건복지부_부모급여(영아수당) 지원.pdf'
 './test_source/보건복지부_노인장기요양보험 사업운영.pdf'
 './test_source/산업통상자원부_에너지바우처.pdf' './test_source/국토교통부_행복주택출자.pdf'
 './test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》.pdf'
 './test_source/「FIS 이슈&포커스」 22-2호 《재정성과관리제도》.pdf'
 './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf']


Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


c:\Users\kim_h\anaconda3\envs\final\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:  11%|█         | 1/9 [00:09<01:16,  9.51s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


...

Processing PDFs:  78%|███████▊  | 7/9 [00:35<00:09,  4.64s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:39<00:04,  4.43s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:43<00:00,  4.86s/it]
